<a href="https://colab.research.google.com/github/Keita-S593/rindoku/blob/main/xfeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考: https://acro-engineer.hatenablog.com/entry/2020/12/15/120000

In [1]:
!pip install git+https://github.com/pfnet-research/xfeat.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pfnet-research/xfeat.git to /tmp/pip-req-build-7tbn1f3p
  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-7tbn1f3p
     |████████████████████████████████| 348 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 13.0 MB/s 
     |████████████████████████████████| 81 kB 6.8 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 112 kB 47.7 MB/s 
     |████████████████████████████████| 147 kB 38.4 MB/s 
     |████████████████████████████████| 49 kB 2.5 MB/s 
  Created wheel for xfeat: filename=xfeat-0.1.1-py3-none-any.whl size=39635 sha256=244a6df3d79680f760011549c7c44a00423cb01e99f0d4f9abd4ae19b2d037ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-3o82tjco/wheels/71/92/2e/4e706daaf3464ec8e3f53b037c5f3f0346ddc1209eda7d5985
  Created wheel for pyperclip: filename=

# インポート

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score
import lightgbm as lgb
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

# 学習データ読み込み

In [3]:
df = pd.read_csv('/content/drive/MyDrive/輪読/titanic/train.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# xfeatを使ってみる

カテゴリカルデータ、数値データをそれぞれ抽出

In [5]:
SelectCategorical().fit_transform(df).head(3)

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S


In [6]:
SelectNumerical().fit_transform(df).head(3)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250


## カテゴリカルデータの処理

ラベルエンコーディング

In [7]:
encoder = Pipeline([SelectCategorical(exclude_cols=["Name", "Ticket"]),LabelEncoder(output_suffix="_label"),])
encoded_df_label = encoder.fit_transform(df)

In [8]:
encoded_df_label.head(3)

,Sex,Cabin,Embarked,Sex_label,Cabin_label,Embarked_label
0,male,NaN,S,0,-1,0
1,female,C85,C,1,0,1
2,female,NaN,S,1,-1,0


ターゲットエンコーディング

ターゲットエンコーディングについて: https://www.codexa.net/target_encoding/

In [9]:
fold = KFold(n_splits=5, shuffle=False)
encoder = TargetEncoder(input_cols=["Cabin"], target_col="Survived", fold=fold, output_suffix="_target")

In [10]:
encoded_df_target = encoder.fit_transform(df)
encoded_df_target[["Survived", "Cabin", "Cabin_target"]].head(3)

,Survived,Cabin,Cabin_target
0,0,NaN,0.303867
1,1,C85,0.303867
2,1,NaN,0.303867


カテゴリカルデータの組み合わせ

In [11]:
encoder = Pipeline([
    SelectCategorical(exclude_cols=["Ticket", "Name"]),
    ConcatCombination(
        output_suffix="", 
        r=2)])
encoded_df = encoder.fit_transform(df)

In [12]:
encoded_df_cat2 = encoder.fit_transform(df)
encoded_df_cat2.head(3)

,Sex,Cabin,Embarked,SexCabin,SexEmbarked,CabinEmbarked
0,male,NaN,S,male_NaN_,maleS,_NaN_S
1,female,C85,C,femaleC85,femaleC,C85C
2,female,NaN,S,female_NaN_,femaleS,_NaN_S


3つのカテゴリカルデータの組み合わせ.r=3にすればok 

In [238]:
encoder = Pipeline([
    SelectCategorical(exclude_cols=["Ticket", "Name"]),
    ConcatCombination(
        # drop_origin=True, 
        output_suffix="", 
        r=3),
])

In [239]:
encoded_df_cat3 = encoder.fit_transform(df)
encoded_df_cat3.head(5)

,Sex,Cabin,Embarked,SexCabinEmbarked
0,male,NaN,S,male_NaN_S
1,female,C85,C,femaleC85C
2,female,NaN,S,female_NaN_S
3,female,C123,S,femaleC123S
4,male,NaN,S,male_NaN_S


## 数値データの処理

数値データの和をとる(積とかでも可能)

In [240]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            # 兄弟/配偶者、両親/子供の数を加算した特徴量を作成する
            input_cols=["SibSp", "Parch"], 
            drop_origin=True, 
            operator="+", 
            r=2,
        ),
    ]
)

In [241]:
encoded_df_add = encoder.fit_transform(df)
encoded_df_add.head()

,SibSpParch_combi
0,1
1,1
2,0
3,1
4,0


Aggregation. ここでは性別ごとに各特徴量の最大値や最小値を取る

In [13]:
aggregated_df, aggregated_cols = aggregation(df,
                     group_key="Sex",
                     group_values=["Age", "Pclass"],
                     agg_methods=["mean", "max"],
                     )

In [15]:
cols_to_show = ["Sex"] + aggregated_cols
aggregated_df[cols_to_show].head(3)

,Sex,agg_mean_Age_grpby_Sex,agg_mean_Pclass_grpby_Sex,agg_max_Age_grpby_Sex,agg_max_Pclass_grpby_Sex
0,male,30.726645,2.389948,80.0,3
1,female,27.915709,2.159236,63.0,3
2,female,27.915709,2.159236,63.0,3


## 特徴量の選択

今までの特徴量を追加

In [244]:
df_encoded = pd.concat([df, encoded_df_label, encoded_df_target, encoded_df_cat2, encoded_df_cat3, encoded_df_add, aggregated_df], axis=1)
df_encoded = df_encoded.loc[:,~df_encoded.columns.duplicated()].drop(['Ticket', 'Name'], axis=1)

encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

df_encoded_label = encoder.fit_transform(df_encoded)

df_encoded = SelectNumerical().fit_transform(pd.concat([df_encoded[SelectNumerical().fit_transform(df_encoded).columns], df_encoded_label], axis=1)).drop('PassengerId', axis=1)

In [245]:
df_encoded.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_label,Cabin_label,Embarked_label,Cabin_target,...,agg_mean_Pclass_grpby_Sex,agg_max_Age_grpby_Sex,agg_max_Pclass_grpby_Sex,Sex,Cabin,Embarked,SexCabin,SexEmbarked,CabinEmbarked,SexCabinEmbarked
0,0,3,22.0,1,0,7.2500,0,-1,0,0.303867,...,2.389948,80.0,3,0,-1,0,0,0,0,0
1,1,1,38.0,1,0,71.2833,1,0,1,0.303867,...,2.159236,63.0,3,1,0,1,1,1,1,1
2,1,3,26.0,0,0,7.9250,1,-1,0,0.303867,...,2.159236,63.0,3,1,-1,0,2,2,0,2


lightgbmののfeatureimportanceに基づいて特徴量を選択

In [246]:
# LightGBMのパラメータを設定する
lgbm_params = {
    "objective": "binary",
    "metric": "binary_error",
}
fit_kwargs = {
    "num_boost_round": 10,
}

# 特徴量選択器を作成する
selector = GBDTFeatureSelector(
    target_col="Survived",
    threshold=0.5,
    lgbm_params=lgbm_params,
    lgbm_fit_kwargs=fit_kwargs,
)


In [247]:
selected_df = selector.fit_transform(df_encoded)
print("Selected columns:", selector._selected_cols)

Selected columns: ['Age', 'Fare', 'Cabin_target', 'SexCabin', 'Pclass', 'Sex_label', 'SibSpParch_combi', 'SexEmbarked', 'SexCabinEmbarked', 'Cabin_label']


# 学習

In [248]:
X = df_encoded[['Age', 'Fare', 'Cabin_target', 'SexCabin', 'Pclass', 'Sex_label', 'SibSpParch_combi', 'SexEmbarked', 'SexCabinEmbarked', 'Cabin']]
y = df_encoded['Survived']

In [249]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=100, stratify=y)

In [250]:
def each_threshold(y, y_pred):
    ths = []
    recalls = []
    precisions = []
    f1s = []
    for th_int in range(1, 99):
        th = th_int / 100. 
        y_bin = (y_pred > th).astype(int)
        tn, fp, fn, tp = confusion_matrix(y, y_bin).flatten()

        rec = tp / (tp + fn + 1e-16)
        pre = tp / (tp + fp + 1e-16)

        ths.append(th)
        recalls.append(rec)
        precisions.append(pre)
        f1s.append(rec * pre / (rec + pre + 1e-16))


    return np.array(ths), np.array(recalls), np.array(precisions), np.array(f1s)

def max_score_with_thre(ths, scores):
    return np.max(scores), ths[(np.argmax(scores))]

def metrics(y, y_pred):
    roc_auc = roc_auc_score(y, y_pred)

    ths, recalls, precisions, f1s = each_threshold(y, y_pred)

    recall, recall_th = max_score_with_thre(ths, recalls)
    precision, precision_th = max_score_with_thre(ths, precisions)
    f1, f1_th = max_score_with_thre(ths, f1s)

    f1mat = confusion_matrix(y, (y_pred > f1_th).astype(int))

    print(f"Max recall is {recall} at  threshold {recall_th}.")
    print(f"Max precision is {precision} at  threshold {precision_th}.")
    print(f"Max f1 is {f1} at  threshold {f1_th}.")
    print("roc auc:", roc_auc)
    print("f1-based confusion matrix:\n", f1mat)

In [251]:
lgb_train = lgb.Dataset(X_train.drop('Cabin', axis=1), y_train)
lgb_eval = lgb.Dataset(X_valid.drop('Cabin', axis=1), y_valid)
 
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 200,
    'learning_rate': 0.003,
    'num_iterations':100,
    'feature_fraction': 0.52,
    'bagging_fraction': 0.79,
    'bagging_freq': 7,
    'verbose': 0
}

cat_list = ['SexCabin', 'Pclass', 'Sex_label', 'SexEmbarked', 'SexCabinEmbarked']

lgb_clf = lgb.train(params,
                lgb_train,
                num_boost_round=5000,
                valid_sets=lgb_eval,
                early_stopping_rounds=1000, 
                categorical_feature=cat_list)

[1]	valid_0's auc: 0.811265
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's auc: 0.855995
[3]	valid_0's auc: 0.895982
[4]	valid_0's auc: 0.882082
[5]	valid_0's auc: 0.904084
[6]	valid_0's auc: 0.90112
[7]	valid_0's auc: 0.901383
[8]	valid_0's auc: 0.901647
[9]	valid_0's auc: 0.906061
[10]	valid_0's auc: 0.907115
[11]	valid_0's auc: 0.90527
[12]	valid_0's auc: 0.908827
[13]	valid_0's auc: 0.906192
[14]	valid_0's auc: 0.910408
[15]	valid_0's auc: 0.910408
[16]	valid_0's auc: 0.909816
[17]	valid_0's auc: 0.908564
[18]	valid_0's auc: 0.908959
[19]	valid_0's auc: 0.907246
[20]	valid_0's auc: 0.908959
[21]	valid_0's auc: 0.908432
[22]	valid_0's auc: 0.907773
[23]	valid_0's auc: 0.906588
[24]	valid_0's auc: 0.905797
[25]	valid_0's auc: 0.906719
[26]	valid_0's auc: 0.90527
[27]	valid_0's auc: 0.903689
[28]	valid_0's auc: 0.903426
[29]	valid_0's auc: 0.903821
[30]	valid_0's auc: 0.90448
[31]	valid_0's auc: 0.904875
[32]	valid_0's auc: 0.904611
[33]	valid_0's auc: 0

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Pclass', 'SexCabin', 'SexCabinEmbarked', 'SexEmbarked', 'Sex_label']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [252]:
metrics(y_valid, lgb_clf.predict(X_valid))

Max recall is 1.0 at  threshold 0.01.
Max precision is 1.0 at  threshold 0.4.
Max f1 is 0.3813559322033898 at  threshold 0.39.
roc auc: 0.9104084321475626
f1-based confusion matrix:
 [[106   4]
 [ 24  45]]


# 推論

In [253]:
df_test = pd.read_csv('/content/drive/MyDrive/輪読/titanic/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/輪読/titanic/gender_submission.csv')

## testデータについても同様の処理

In [254]:
encoder = Pipeline([
    SelectCategorical(exclude_cols=["Name", "Ticket"]),
    LabelEncoder(output_suffix="_label"),
])

encoded_df_label = encoder.fit_transform(df_test)

In [255]:
len(encoded_df_label)

418

In [256]:
encoder = Pipeline([
    SelectCategorical(exclude_cols=["Ticket", "Name"]),
    ConcatCombination(
        # drop_origin=True, 
        output_suffix="", 
        r=2),
])

encoded_df_cat2 = encoder.fit_transform(df_test)

In [257]:
len(encoded_df_cat2)

418

In [258]:
encoder = Pipeline([
    SelectCategorical(exclude_cols=["Ticket", "Name"]),
    ConcatCombination(
        # drop_origin=True, 
        output_suffix="", 
        r=3),
])

encoded_df_cat3 = encoder.fit_transform(df_test)

In [259]:
len(encoded_df_cat3)

418

In [260]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            # 兄弟/配偶者、両親/子供の数を加算した特徴量を作成する
            input_cols=["SibSp", "Parch"], 
            drop_origin=True, 
            operator="+", 
            r=2,
        ),
    ]
)

encoded_df_add = encoder.fit_transform(df_test)

In [261]:
aggregated_df, aggregated_cols = aggregation(df_test,
                     group_key="Sex",
                     group_values=["Age", "Pclass"],
                     agg_methods=["mean", "max"],
                     )

In [262]:
len(aggregated_df)

418

In [263]:
df_encoded = pd.concat([df_test, encoded_df_label, encoded_df_cat2, encoded_df_cat3, encoded_df_add, aggregated_df], axis=1)
df_encoded = df_encoded.loc[:,~df_encoded.columns.duplicated()].drop(['Ticket', 'Name'], axis=1)

encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

df_encoded_label = encoder.fit_transform(df_encoded)

X_test = SelectNumerical().fit_transform(pd.concat([df_encoded[SelectNumerical().fit_transform(df_encoded).columns], df_encoded_label], axis=1)).drop('PassengerId', axis=1)
print(len(X_test))
X_test = pd.merge(X_test, X[['Cabin', 'Cabin_target']].drop_duplicates(subset='Cabin'), on='Cabin')[['Age', 'Fare', 'Cabin_target', 'SexCabin', 'Pclass', 'Sex_label', 'SibSpParch_combi', 'SexEmbarked', 'SexCabinEmbarked']]

418


In [264]:
X_test

,Age,Fare,Cabin_target,SexCabin,Pclass,Sex_label,SibSpParch_combi,SexEmbarked,SexCabinEmbarked
0,34.5,7.8292,0.303867,0,3,0,0,0,0
1,47.0,7.0000,0.303867,1,3,1,1,1,1
2,62.0,9.6875,0.303867,0,2,0,0,0,0
3,27.0,8.6625,0.303867,0,3,0,0,2,2
4,22.0,12.2875,0.303867,1,3,1,2,1,1
...,...,...,...,...,...,...,...,...,...
413,48.0,79.2000,0.294872,80,1,1,2,4,84
414,30.0,164.8667,1.000000,81,1,1,0,1,85
415,43.0,27.7208,0.294872,82,1,0,1,5,86
416,20.0,13.8625,1.000000,83,2,0,0,5,87


閾値はf1が最大になる最大になる0.39を採用

In [265]:
y_pred_test = (lgb_clf.predict(X_test) >0.39)*1

In [266]:
sub['Survived'] = y_pred_test
sub['Survived'].value_counts()

0    282
1    136
Name: Survived, dtype: int64

In [268]:
sub.to_csv('/content/drive/MyDrive/輪読/submit/submit_xfeat.csv', index=False)

スコアは0.7655